In [1]:
import os
os.chdir("../..")
import dotenv


dotenv.load_dotenv(override=True)

print(os.getcwd(), os.getenv("HF_HOME"))

import json
import time
from concurrent.futures import ProcessPoolExecutor, as_completed
from functools import partial
from hashlib import sha256
from pathlib import Path

import datasets
import matplotlib.pyplot as plt
import pandas as pd
import tqdm

/data1/xhuan192/codes/med-sipf cache/


/data1/xhuan192/misc/miniconda3/envs/xiaoke-med_sipf-1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.getcwd()

'/data1/xhuan192/codes/med-sipf'

In [3]:
def remove_boxed(s, left="\\boxed"):
    original_s = s
    left = left + "{"
    try:
        assert s[: len(left)] == left
        assert s[-1] == "}"
        answer = s[len(left) : -1]
        if "=" in answer:
            answer = answer.split("=")[-1].lstrip(" ")
        return answer
    except Exception:
        return original_s


def last_boxed_only_string(string, left="\\boxed"):
    idx = string.rfind("\\boxed")
    if idx < 0:
        idx = string.rfind("\\fbox")
        if idx < 0:
            return string
    i = idx
    right_brace_idx = None
    num_left_braces_open = 0
    while i < len(string):
        if string[i] == "{":
            num_left_braces_open += 1
        if string[i] == "}":
            num_left_braces_open -= 1
            if num_left_braces_open == 0:
                right_brace_idx = i
                break
        i += 1

    if right_brace_idx is None:
        retval = string
    else:
        retval = string[idx : right_brace_idx + 1]

    return retval


def extract_answer(answer):
    # Try to extract content inside \boxed{}
    answer = remove_boxed(last_boxed_only_string(answer))
    return remove_boxed(last_boxed_only_string(answer, "\\text"), "\\text")


def eval_sample(x, prefix=""):
    model_answer_string = x["distilled_answer_string"]
    answer_letter = x["answer_letter"]
    answer_idx = x["answer_idx"]
    answer_string = x["answer_string"]

    extracted_answer_string = extract_answer(model_answer_string)

    try:
        if extracted_answer_string.strip().lower() == answer_string.strip().lower():
            correct = True
        elif (
            extracted_answer_string[0].strip().lower() == answer_letter.strip().lower()
        ):
            correct = True
        else:
            correct = False
    except Exception as e:
        print(f"Error: {e}\n{model_answer_string}\n{extracted_answer_string}")
        correct = False
    return {
        f"{prefix}extracted_answer_string": extracted_answer_string,
        f"{prefix}model_answer_string": model_answer_string,
        f"{prefix}correct": correct,
    }


In [4]:
def eval_correctness(path, split, prefix):
    dataset = datasets.load_dataset(path,split=split)

    mapped_dataset = dataset.map(partial(eval_sample, prefix=prefix), keep_in_memory=True, remove_columns=dataset.column_names)
    return mapped_dataset

path_list = ["mmqm/m196k-dedup-decon-filter_easy-r1"]
prefix_list = ["r1-"]
split="train"

eval_correctness_list = []

for path, prefix in zip(path_list, prefix_list):

    mapped_dataset = eval_correctness(path, split, prefix)
    eval_correctness_list.append(mapped_dataset)

    correct_list=mapped_dataset[f"{prefix}correct"]
    print(f"path: {path}")
    print(f"correct: {sum(correct_list)}")
    print(f"total: {len(correct_list)}")
    print(f"accuracy: {sum(correct_list)/len(correct_list)}")

Map:   4%|▍         | 1699/37815 [00:00<00:04, 8538.98 examples/s]

Error: string index out of range


Error: string index out of range


Error: string index out of range




Map:  26%|██▌       | 9871/37815 [00:01<00:02, 9576.47 examples/s]

Error: string index out of range


Error: string index out of range


Error: string index out of range




Map:  29%|██▊       | 10835/37815 [00:01<00:02, 9591.76 examples/s]

...

Error: string index out of range


Error: string index out of range




Map:  93%|█████████▎| 35034/37815 [00:03<00:00, 9486.81 examples/s]

Error: string index out of range




Map: 100%|██████████| 37815/37815 [00:04<00:00, 9207.54 examples/s]


path: mmqm/m196k-dedup-decon-filter_easy-r1
correct: 23504
total: 37815
accuracy: 0.6215522940632024


```
path: mmqm/m196k-dedup-decon-filter_easy-r1-250311
correct: 23494
total: 37786
accuracy: 0.6217646747472609

path: mmqm/m196k-dedup-decon-filter_easy-r1
correct: 23504
total: 37815
accuracy: 0.6215522940632024
```

In [ ]:
path="mmqm/m196k-dedup-decon-filter_easy-r1"
split="train"

raw_dataset = datasets.load_dataset(path,split=split)
merged_dataset = datasets.concatenate_datasets([raw_dataset] + eval_correctness_list, axis=1)

In [ ]:
def filter_false(x, prefix_list=["r1-"]):
    for prefix in prefix_list:
        if x[f"{prefix}correct"]:
            return True
    return False

filtered_dataset = merged_dataset.filter(filter_false)
filtered_dataset

Filter: 100%|██████████| 37815/37815 [00:00<00:00, 38039.77 examples/s]


In [8]:
filtered_dataset.push_to_hub("mmqm/m196k-dedup-decon-filter_easy-r1-filter_wrong")

Uploading the dataset shards: 100%|██████████| 1/1 [00:08<00:00,  8.29s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/mmqm/m196k-dedup-decon-filter_easy-r1-filter_wrong/commit/5102cab42552c317e15aa7bd549195e4f936a5c2', commit_message='Upload dataset', commit_description='', oid='5102cab42552c317e15aa7bd549195e4f936a5c2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mmqm/m196k-dedup-decon-filter_easy-r1-filter_wrong', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mmqm/m196k-dedup-decon-filter_easy-r1-filter_wrong'), pr_revision=None, pr_num=None)